# Convolutional Network In TensorFlow

### © Jubeen Shah 2018

Hey there! Welcome to `J.S Codes` jupyter notebooks for TensorFlow!

Thanks to [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples) for providing the original TensorFlow model on which this segment is based.

## Dataset
We've seen this section of the code from the previous notebooks. We will be importing the MNIST data set using a convenient TensorFlow function to batch, scale, and One-Hot encode the data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot = True, reshape = False )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [31]:
#parameters
learning_rate = 0.001
epochs = 10
batch_size = 128

In [32]:
#Number of samples to calculate the validation and accuracy
test_valid_size = 256

In [33]:
#Network Parameters
n_classes = 10
dropout = 0.75 

In [34]:
import tensorflow as tf

## Weights and Biases

In [35]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2' : tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out' : tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

## Convolutions

<img src = 'images/01.gif'>
Convolution with 3×3 Filter. Source: http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution

The above is an example of a convolution with a 3x3 filter and a stride of 1 being applied to data with a range of 0 to 1. The convolution for each 3x3 section is calculated against the weight, [[1, 0, 1], [0, 1, 0], [1, 0, 1]], then a bias is added to create the convolved feature on the right. In this case, the bias is zero. In TensorFlow, this is all done using `tf.nn.conv2d()` and `tf.nn.bias_add()`.

In [36]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(input=x, filter=W, strides=[1,strides, strides,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

The `tf.nn.conv2d()` function computes the convolution against weight W as shown above.

In TensorFlow, `strides` is an array of `4 elements`; the `first element in this array indicates the stride for batch` and `last element indicates stride for features`. It's good practice to remove the batches or features you want to skip from the data set rather than use a stride to skip them. You can always set the first and last element to 1 in strides in order to use all batches and features.

The middle two elements are the strides for height and width respectively. I've mentioned stride as one number because you usually have a square stride where `height = width`. When someone says they are using a stride of 3, they usually mean tf.nn.conv2d(x, W, strides=[1, 3, 3, 1]).

To make life easier, the code is using tf.nn.bias_add() to add the bias. Using tf.add() doesn't work when the tensors aren't the same shape.

## Max Pooling

<img src = 'images/02.jpeg'> 

The above is an example of max pooling with a 2x2 filter and stride of 2. The left square is the input and the right square is the output. The four 2x2 colors in input represents each time the filter was applied to create the max on the right side. For example, [[1, 1], [5, 6]] becomes 6 and [[3, 2], [1, 2]] becomes 3.

In [37]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(value=x, ksize=[1,k,k,1], strides=[1,k,k,1], padding = 'SAME')

The `tf.nn.max_pool()` function does exactly what you would expect, it performs max pooling with the `ksize` parameter as the size of the filter.

## Model
<img src = 'images/03.png'>

In the code below, we're creating 3 layers alternating between convolutions and max pooling followed by a fully connected and output layer. The transformation of each layer to new dimensions are shown in the comments. For example, the first layer shapes the images from 28x28x1 to 28x28x32 in the convolution step. Then next step applies max pooling, turning each sample into 14x14x32. All the layers are applied from conv1 to output, producing 10 class predictions.

In [38]:
def conv_net(x, weights, biases, dropout):
    #Layer 1 -- 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'],biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    #Layer 2 -- 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    
    #fully connected layer -- 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #output Layer -- class prediction -- 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Session

Let's now run it

In [39]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 610092.2500 Validation Accuracy: 0.097656
Epoch  1, Batch   2 -Loss: 178664.6875 Validation Accuracy: 0.093750
Epoch  1, Batch   3 -Loss: 14733.4717 Validation Accuracy: 0.074219
Epoch  1, Batch   4 -Loss:  5400.4629 Validation Accuracy: 0.074219
Epoch  1, Batch   5 -Loss:  2647.8459 Validation Accuracy: 0.066406
Epoch  1, Batch   6 -Loss:   890.1825 Validation Accuracy: 0.101562
Epoch  1, Batch   7 -Loss:   210.5814 Validation Accuracy: 0.066406
Epoch  1, Batch   8 -Loss:   116.4816 Validation Accuracy: 0.101562
Epoch  1, Batch   9 -Loss:   110.5924 Validation Accuracy: 0.101562
Epoch  1, Batch  10 -Loss:   119.5626 Validation Accuracy: 0.101562
Epoch  1, Batch  11 -Loss:   135.5456 Validation Accuracy: 0.101562
Epoch  1, Batch  12 -Loss:   122.0403 Validation Accuracy: 0.101562
Epoch  1, Batch  13 -Loss:    90.5455 Validation Accuracy: 0.101562
Epoch  1, Batch  14 -Loss:   110.6505 Validation Accuracy: 0.101562
Epoch  1, Batch  15 -Loss:   135.1866 Validati

Epoch  1, Batch 122 -Loss:    80.4591 Validation Accuracy: 0.101562
Epoch  1, Batch 123 -Loss:    66.0796 Validation Accuracy: 0.101562
Epoch  1, Batch 124 -Loss:    78.4911 Validation Accuracy: 0.101562
Epoch  1, Batch 125 -Loss:    96.3883 Validation Accuracy: 0.101562
Epoch  1, Batch 126 -Loss:    77.1676 Validation Accuracy: 0.101562
Epoch  1, Batch 127 -Loss:    83.1795 Validation Accuracy: 0.101562
Epoch  1, Batch 128 -Loss:    65.4168 Validation Accuracy: 0.101562
Epoch  1, Batch 129 -Loss:   105.3551 Validation Accuracy: 0.101562
Epoch  1, Batch 130 -Loss:    89.1393 Validation Accuracy: 0.101562
Epoch  1, Batch 131 -Loss:    87.9632 Validation Accuracy: 0.101562
Epoch  1, Batch 132 -Loss:    92.1196 Validation Accuracy: 0.101562
Epoch  1, Batch 133 -Loss:    74.7129 Validation Accuracy: 0.101562
Epoch  1, Batch 134 -Loss:    75.6945 Validation Accuracy: 0.101562
Epoch  1, Batch 135 -Loss:    94.9135 Validation Accuracy: 0.101562
Epoch  1, Batch 136 -Loss:    90.1231 Validation

Epoch  1, Batch 243 -Loss:    87.8553 Validation Accuracy: 0.117188
Epoch  1, Batch 244 -Loss:    81.0792 Validation Accuracy: 0.117188
Epoch  1, Batch 245 -Loss:    62.8982 Validation Accuracy: 0.117188
Epoch  1, Batch 246 -Loss:    62.5890 Validation Accuracy: 0.117188
Epoch  1, Batch 247 -Loss:    73.3302 Validation Accuracy: 0.117188
Epoch  1, Batch 248 -Loss:    67.5462 Validation Accuracy: 0.117188
Epoch  1, Batch 249 -Loss:    48.2085 Validation Accuracy: 0.117188
Epoch  1, Batch 250 -Loss:    68.7707 Validation Accuracy: 0.117188
Epoch  1, Batch 251 -Loss:    66.2048 Validation Accuracy: 0.117188
Epoch  1, Batch 252 -Loss:    50.3971 Validation Accuracy: 0.117188
Epoch  1, Batch 253 -Loss:    68.9082 Validation Accuracy: 0.066406
Epoch  1, Batch 254 -Loss:    80.2452 Validation Accuracy: 0.066406
Epoch  1, Batch 255 -Loss:    54.5761 Validation Accuracy: 0.066406
Epoch  1, Batch 256 -Loss:    56.5926 Validation Accuracy: 0.066406
Epoch  1, Batch 257 -Loss:    61.9331 Validation

Epoch  1, Batch 364 -Loss:    41.2031 Validation Accuracy: 0.117188
Epoch  1, Batch 365 -Loss:    65.1937 Validation Accuracy: 0.117188
Epoch  1, Batch 366 -Loss:    56.7794 Validation Accuracy: 0.117188
Epoch  1, Batch 367 -Loss:    52.7915 Validation Accuracy: 0.117188
Epoch  1, Batch 368 -Loss:    60.9599 Validation Accuracy: 0.117188
Epoch  1, Batch 369 -Loss:    54.7237 Validation Accuracy: 0.117188
Epoch  1, Batch 370 -Loss:    46.6038 Validation Accuracy: 0.117188
Epoch  1, Batch 371 -Loss:    47.6184 Validation Accuracy: 0.117188
Epoch  1, Batch 372 -Loss:    61.4634 Validation Accuracy: 0.117188
Epoch  1, Batch 373 -Loss:    56.5803 Validation Accuracy: 0.117188
Epoch  1, Batch 374 -Loss:    73.0148 Validation Accuracy: 0.117188
Epoch  1, Batch 375 -Loss:    64.8849 Validation Accuracy: 0.117188
Epoch  1, Batch 376 -Loss:    62.4896 Validation Accuracy: 0.117188
Epoch  1, Batch 377 -Loss:    59.6728 Validation Accuracy: 0.117188
Epoch  1, Batch 378 -Loss:    50.1475 Validation

Epoch  2, Batch  56 -Loss:    41.8336 Validation Accuracy: 0.117188
Epoch  2, Batch  57 -Loss:    66.2166 Validation Accuracy: 0.117188
Epoch  2, Batch  58 -Loss:    35.0554 Validation Accuracy: 0.117188
Epoch  2, Batch  59 -Loss:    46.0436 Validation Accuracy: 0.117188
Epoch  2, Batch  60 -Loss:    52.2212 Validation Accuracy: 0.117188
Epoch  2, Batch  61 -Loss:    47.2080 Validation Accuracy: 0.117188
Epoch  2, Batch  62 -Loss:    52.7175 Validation Accuracy: 0.117188
Epoch  2, Batch  63 -Loss:    48.2649 Validation Accuracy: 0.117188
Epoch  2, Batch  64 -Loss:    38.5711 Validation Accuracy: 0.117188
Epoch  2, Batch  65 -Loss:    29.1993 Validation Accuracy: 0.117188
Epoch  2, Batch  66 -Loss:    58.0199 Validation Accuracy: 0.117188
Epoch  2, Batch  67 -Loss:    44.0304 Validation Accuracy: 0.117188
Epoch  2, Batch  68 -Loss:    49.5612 Validation Accuracy: 0.117188
Epoch  2, Batch  69 -Loss:    38.6348 Validation Accuracy: 0.117188
Epoch  2, Batch  70 -Loss:    61.4806 Validation

...

Epoch  9, Batch 320 -Loss:     2.8478 Validation Accuracy: 0.117188
Epoch  9, Batch 321 -Loss:     2.7284 Validation Accuracy: 0.117188
Epoch  9, Batch 322 -Loss:     2.5276 Validation Accuracy: 0.117188
Epoch  9, Batch 323 -Loss:     2.6494 Validation Accuracy: 0.117188
Epoch  9, Batch 324 -Loss:     2.7693 Validation Accuracy: 0.117188
Epoch  9, Batch 325 -Loss:     2.8134 Validation Accuracy: 0.117188
Epoch  9, Batch 326 -Loss:     2.8220 Validation Accuracy: 0.117188
Epoch  9, Batch 327 -Loss:     2.8060 Validation Accuracy: 0.117188
Epoch  9, Batch 328 -Loss:     2.5848 Validation Accuracy: 0.117188
Epoch  9, Batch 329 -Loss:     2.7404 Validation Accuracy: 0.117188
Epoch  9, Batch 330 -Loss:     2.7210 Validation Accuracy: 0.117188
Epoch  9, Batch 331 -Loss:     2.7788 Validation Accuracy: 0.117188
Epoch  9, Batch 332 -Loss:     2.8029 Validation Accuracy: 0.117188
Epoch  9, Batch 333 -Loss:     2.6257 Validation Accuracy: 0.117188
Epoch  9, Batch 334 -Loss:     2.8684 Validation

Epoch 10, Batch  12 -Loss:     2.6607 Validation Accuracy: 0.117188
Epoch 10, Batch  13 -Loss:     2.7545 Validation Accuracy: 0.117188
Epoch 10, Batch  14 -Loss:     2.7965 Validation Accuracy: 0.117188
Epoch 10, Batch  15 -Loss:     2.7494 Validation Accuracy: 0.117188
Epoch 10, Batch  16 -Loss:     2.7332 Validation Accuracy: 0.117188
Epoch 10, Batch  17 -Loss:     2.7368 Validation Accuracy: 0.117188
Epoch 10, Batch  18 -Loss:     2.6611 Validation Accuracy: 0.117188
Epoch 10, Batch  19 -Loss:     2.8490 Validation Accuracy: 0.117188
Epoch 10, Batch  20 -Loss:     2.6235 Validation Accuracy: 0.117188
Epoch 10, Batch  21 -Loss:     2.7032 Validation Accuracy: 0.117188
Epoch 10, Batch  22 -Loss:     2.6823 Validation Accuracy: 0.117188
Epoch 10, Batch  23 -Loss:     2.6288 Validation Accuracy: 0.117188
Epoch 10, Batch  24 -Loss:     2.7244 Validation Accuracy: 0.117188
Epoch 10, Batch  25 -Loss:     2.7041 Validation Accuracy: 0.117188
Epoch 10, Batch  26 -Loss:     2.6768 Validation

Epoch 10, Batch 133 -Loss:     2.5586 Validation Accuracy: 0.117188
Epoch 10, Batch 134 -Loss:     2.6749 Validation Accuracy: 0.117188
Epoch 10, Batch 135 -Loss:     2.7525 Validation Accuracy: 0.117188
Epoch 10, Batch 136 -Loss:     2.6229 Validation Accuracy: 0.117188
Epoch 10, Batch 137 -Loss:     2.5936 Validation Accuracy: 0.117188
Epoch 10, Batch 138 -Loss:     2.6954 Validation Accuracy: 0.117188
Epoch 10, Batch 139 -Loss:     2.6685 Validation Accuracy: 0.117188
Epoch 10, Batch 140 -Loss:     2.6811 Validation Accuracy: 0.117188
Epoch 10, Batch 141 -Loss:     2.5495 Validation Accuracy: 0.117188
Epoch 10, Batch 142 -Loss:     2.5772 Validation Accuracy: 0.117188
Epoch 10, Batch 143 -Loss:     2.5805 Validation Accuracy: 0.117188
Epoch 10, Batch 144 -Loss:     2.5439 Validation Accuracy: 0.117188
Epoch 10, Batch 145 -Loss:     2.6136 Validation Accuracy: 0.117188
Epoch 10, Batch 146 -Loss:     2.4759 Validation Accuracy: 0.117188
Epoch 10, Batch 147 -Loss:     2.7035 Validation

Epoch 10, Batch 254 -Loss:     2.6811 Validation Accuracy: 0.066406
Epoch 10, Batch 255 -Loss:     2.7555 Validation Accuracy: 0.117188
Epoch 10, Batch 256 -Loss:     2.6469 Validation Accuracy: 0.066406
Epoch 10, Batch 257 -Loss:     2.7154 Validation Accuracy: 0.066406
Epoch 10, Batch 258 -Loss:     2.5304 Validation Accuracy: 0.066406
Epoch 10, Batch 259 -Loss:     2.6575 Validation Accuracy: 0.066406
Epoch 10, Batch 260 -Loss:     2.7100 Validation Accuracy: 0.117188
Epoch 10, Batch 261 -Loss:     2.9437 Validation Accuracy: 0.117188
Epoch 10, Batch 262 -Loss:     2.6364 Validation Accuracy: 0.117188
Epoch 10, Batch 263 -Loss:     2.7477 Validation Accuracy: 0.117188
Epoch 10, Batch 264 -Loss:     2.7015 Validation Accuracy: 0.117188
Epoch 10, Batch 265 -Loss:     2.7189 Validation Accuracy: 0.117188
Epoch 10, Batch 266 -Loss:     2.7899 Validation Accuracy: 0.117188
Epoch 10, Batch 267 -Loss:     2.6766 Validation Accuracy: 0.117188
Epoch 10, Batch 268 -Loss:     2.8281 Validation

Epoch 10, Batch 375 -Loss:     2.7526 Validation Accuracy: 0.117188
Epoch 10, Batch 376 -Loss:     2.6399 Validation Accuracy: 0.117188
Epoch 10, Batch 377 -Loss:     2.6470 Validation Accuracy: 0.117188
Epoch 10, Batch 378 -Loss:     2.5833 Validation Accuracy: 0.117188
Epoch 10, Batch 379 -Loss:     2.6469 Validation Accuracy: 0.117188
Epoch 10, Batch 380 -Loss:     2.7901 Validation Accuracy: 0.117188
Epoch 10, Batch 381 -Loss:     2.8100 Validation Accuracy: 0.117188
Epoch 10, Batch 382 -Loss:     2.6179 Validation Accuracy: 0.117188
Epoch 10, Batch 383 -Loss:     2.6761 Validation Accuracy: 0.117188
Epoch 10, Batch 384 -Loss:     2.6673 Validation Accuracy: 0.117188
Epoch 10, Batch 385 -Loss:     2.6522 Validation Accuracy: 0.117188
Epoch 10, Batch 386 -Loss:     2.6862 Validation Accuracy: 0.117188
Epoch 10, Batch 387 -Loss:     2.6012 Validation Accuracy: 0.117188
Epoch 10, Batch 388 -Loss:     2.7497 Validation Accuracy: 0.117188
Epoch 10, Batch 389 -Loss:     2.5516 Validation